# 📦 Tutorial 1: Data Extraction from DXF Files

## 🎯 What You'll Learn
- How building data is stored in DXF files
- Extract rooms, doors, and walls from CAD files
- Validate data with Pydantic schemas
- Visualize extracted floor plans

---


## 🏗️ What is a DXF File?

DXF (Drawing Exchange Format) is a CAD data file format used for:
- AutoCAD drawings
- Architectural plans  
- Technical drawings

It contains geometric entities like:
- **LINES** (walls)
- **POLYLINES** (room boundaries)
- **INSERT** (blocks like doors)
- **TEXT** (labels)


## 📥 Step 1: Load Example Data

Instead of parsing a real DXF (complex), we'll use pre-extracted JSON.


In [ ]:
import json
from pathlib import Path
from pprint import pprint

# Load example project
with open('data/extracted/I01.4_mock_data_converted.json', 'r') as f:
    project_data = json.load(f)

print("✅ Loaded project:", project_data['metadata']['project_name'])
print(f"   Level: {project_data['metadata']['level_name']}")


In [ ]:
# Inspect rooms
rooms = project_data['rooms']
print(f"Found {len(rooms)} rooms:")
for room in rooms:
    print(f"📍 {room['name']} ({room['id']}) - {len(room['boundary'])} boundary points")

# Inspect doors  
doors = project_data['doors']
print(f"\nFound {len(doors)} doors:")
for door in doors:
    width_cm = door['width'] * 100
    egress = "✅ Egress" if door['is_egress'] else "❌ Interior"
    print(f"🚪 {door['id']}: {width_cm:.0f}cm - {egress}")


In [ ]:
# Visualize floor plan
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 8))

# Plot rooms
for room in rooms:
    boundary = room['boundary']
    xs = [p[0] for p in boundary]
    ys = [p[1] for p in boundary]
    
    ax.plot(xs, ys, 'b-', linewidth=2)
    ax.fill(xs, ys, alpha=0.1, color='blue')
    
    # Add room label
    center_x = sum(xs) / len(xs)
    center_y = sum(ys) / len(ys)
    ax.text(center_x, center_y, room['name'], 
            ha='center', va='center', fontsize=10, fontweight='bold')

# Plot doors
for door in doors:
    x, y = door['position']
    color = 'green' if door['is_egress'] else 'orange'
    ax.scatter(x, y, s=200, c=color, marker='s', zorder=5)
    ax.text(x, y+0.5, door['id'], ha='center', fontsize=8)

ax.set_xlabel('X (meters)')
ax.set_ylabel('Y (meters)')
ax.set_title('PCI - Extinción Automática - Floor Plan', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.axis('equal')

plt.tight_layout()
plt.show()

print("🟦 Blue = Rooms")
print("🟩 Green = Egress doors") 
print("🟧 Orange = Interior doors")


## 🎯 Summary

In this tutorial, you learned:

1. ✅ DXF files contain geometric building data
2. ✅ Extract rooms, doors, walls from DXF
3. ✅ Validate data with Pydantic schemas
4. ✅ Visualize floor plans with matplotlib

**Next**: Tutorial 2 - Calculations (areas, routes)
